In [1]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline

# Progress bar
from tqdm.auto import trange, tqdm

In [10]:
import webbpsf_ext, pynrc
from astropy.io import fits

In [53]:
apn = 'NRCB5_SUB160'

nrc = pynrc.NIRCam(filter='F410M', apname=apn, autogen_coeffs=False)
nrc.aperturename = apn
nrc.fov_pix = 155

nrc.npsf = 20
nrc.ndeg = 10

nrc.load_wss_opd_by_date(date='2022-07-29', verbose=False)

[   webbpsf:INFO] NIRCam aperture name updated to NRCA1_FULL
[   webbpsf:INFO] NIRCam pixel scale switched to 0.063000 arcsec/pixel for the long wave channel.
[   webbpsf:INFO] NIRCam aperture name updated to NRCA5_FULL
[   webbpsf:INFO] NIRCam aperture name updated to NRCB5_FULL
[     pynrc:INFO] RAPID readout mode selected.
[     pynrc:INFO] Setting ngroup=1, nf=1, nd1=0, nd2=0, nd3=0.
[     pynrc:INFO] Initializing SCA 490/B5
[     pynrc:INFO] Suggested SIAF aperture name: NRCB5_SUB160P
[   webbpsf:INFO] NIRCam aperture name updated to NRCB5_SUB160
[   webbpsf:INFO] NIRCam aperture name updated to NRCA1_FULL
[   webbpsf:INFO] NIRCam aperture name updated to NRCA3_FULL
[   webbpsf:INFO] NIRCam aperture name updated to NRCA3_FP1
[     poppy:INFO] OPD from /home/jarronl/NIRCam/webbpsf-data/NIRCam/OPD/wss_target_phase_fp1.fits: Loaded OPD from /home/jarronl/NIRCam/webbpsf-data/NIRCam/OPD/wss_target_phase_fp1.fits
[     poppy:INFO] No info supplied on amplitude transmission; assuming uni

In [54]:
# Generate the PSF coefficients
res = nrc.gen_psf_coeff(force=True, save=False, return_results=True, return_extras=True)
psf_coeff, psf_coeff_header, extras_dict = res

# Set object attributes
nrc.psf_coeff = psf_coeff
nrc.psf_coeff_header = psf_coeff_header

[webbpsf_ext:INFO] Generating but not saving PSF coefficient


Monochromatic PSFs:   0%|          | 0/20 [00:00<?, ?it/s]

[webbpsf_ext:INFO] Took 5.81 seconds to generate WebbPSF images


In [55]:
sp = pynrc.stellar_spectrum('G2V', 15, 'vegamag', nrc.bandpass)
hdul_im = nrc.calc_psf_from_coeff(sp, return_hdul=True, return_oversample=False)

In [56]:
hdul_im.writeto('NRCB5_SUB160_F410M_G2V_2022-07-29.fits', overwrite=True)

In [62]:
# Downsample to detector pixels
from webbpsf_ext.image_manip import frebin

psf_coeff_det = frebin(psf_coeff, scale=1/nrc.oversample)
hdu = fits.PrimaryHDU(psf_coeff_det, header=hdul_im[0].header)
hdu.writeto('NRCB5_SUB160_F410M_psf-coeff-lib_2022-07-29.fits', overwrite=True)